<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/01-pyAMPACT_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Introduction</h1>

In [1]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact==0.4.6
    import pyampact
import pandas as pd
import numpy as np
import librosa

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 263 (delta 58), reused 17 (delta 14), pack-reused 177
Receiving objects: 100% (263/263), 17.37 MiB | 21.20 MiB/s, done.
Resolving deltas: 100% (119/119), done.
Importing libraries...


Load audio and symbolic files. Audio files can be any format supported by librosa (wav, mp3, flac, ogg, etc.) and symbolic files can be any format supported by music21 (kern, xml, mei, midi, abc, etc.).

The audio file should be read with librosa's `load` function and the symbolic file should be read into a Score object with pyAMPACT's `Score` function.

In [2]:
audio_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.wav'
midi_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn'

y, original_sr = librosa.load(audio_file)

piece = pyampact.Score(midi_file)


	Detected and imported these spine types:
		 harm function 



DO WE WANT TO KEEP THE OPTION TO PUT IN NEW MEANS/COVARS OR SHOULD WE JUST HARDCODE THEM? IN EITHER CASE WE PROBABLY DON'T WANT THIS IN THE ALPHA RELEASE WHERE WE ARE JUST DOING DTW.

Load default means and covariances for alignment refinement (these can be replaced with instrument specific ones)

In [3]:
means = pd.read_csv('/content/pyAMPACTtutorials/test_files/means.csv', sep=' ').values
covars = pd.read_csv('/content/pyAMPACTtutorials/test_files/covars.csv', sep=' ').values

Run the alignment with pyampact's `run_alignment` function.

You can specify the target sample rate (target_sr), window size in milliseconds (win_ms), the hop size (hop_length) as well as arguments that dictate the way that the symbolic file is represented as a spectrogram-like mask: the bin width of each note/harmonic (width) and the number of harmonics (n_harm).

CAN WE REMOVE THE PRINT-OUTS AND AUTOMATIC VISUALIZATION IN RUN_ALIGNMENT?

In [4]:
target_sr = 4000
win_ms = 100
hop_length = 32
width = 3
n_harm = 3

res, dtw, spec, nmat = pyampact.run_alignment(
    y, original_sr, piece, means, covars, piece.nmats(), width, target_sr, n_harm, win_ms, hop_length)

/usr/local/lib/python3.10/dist-packages/pyampact/alignmentUtils.py:452: ComplexWarning: Casting complex values to real discards the imaginary part
  S[r, :] = np.sqrt(M[:, r] @ D) / nDc


Visualize the alignment

In [5]:
times = []
freqs = []
pyampact.alignment_visualiser(spec, times, freqs, 1)

Estimate performance paramters.

In this example we are estimating F0 and power values for each note. See XXX notebook for examples of all of the performance parameters that can be estimated by pyAMPACT.

In [6]:
# Calculate an instananeous frequency-gram of the audio signal
freqs, times, mags, f0_values, mags_mat = pyampact.ifgram(audiofile=audio_file, tsr=target_sr, win_ms=win_ms)
mags_db = librosa.amplitude_to_db(mags, ref=np.max)

# Estimate F0 values
f0_values, sig_pwr = pyampact.calculate_f0_est(audio_file, hop_length, win_ms, target_sr)
sig_pwr = mags ** 2 # power of signal, magnitude/amplitude squared

# Prune NaN and zero values from estimated F0 and power
f0_values = f0_values[~np.isnan(f0_values)]
sig_pwr = sig_pwr[sig_pwr != 0]

Export estimated performance parameters to a .json file

In [7]:
# def data_compilation(f0_values, sig_pwr, mag_mat, nmat, target_sr, hop_length, y, audio_file_path=None, export_path=None):

pyampact.data_compilation(f0_values, sig_pwr, mags_mat, nmat, target_sr, hop_length, y, audio_file)

{'Part-1':                  MEASURE     ONSET  DURATION    PART  MIDI  ONSET_SEC  \
XML_ID                                                                  
133935999646640        1   0.00000   2.00000  Part-1  79.0    0.00000   
133935999643424        1   2.00000   0.25000  Part-1  77.0    2.00000   
133935999633296        1   2.25000   0.25000  Part-1  76.0    2.25000   
133935999636032        1   2.50000   0.25000  Part-1  74.0    2.50000   
133935999632288        1   2.75000   0.25000  Part-1  72.0    2.75000   
133935999645296        2   3.00000   0.50000  Part-1  79.0    3.00000   
133935999641984        2   3.50000   0.50000  Part-1  77.0    3.50000   
133935999631904        2   4.00000   1.50000  Part-1  74.0    4.00000   
133935999644768        2   5.50000   0.50000  Part-1  77.0    5.50000   
133935999641072        3   6.00000   0.33333  Part-1  76.0    6.00000   
133935999636464        3   6.33333   0.33333  Part-1  79.0    6.33333   
133935999640304        3   6.66667   0.3

({'Part-1':                  MEASURE     ONSET  DURATION    PART  MIDI  ONSET_SEC  \
  XML_ID                                                                  
  133935999646640        1   0.00000   2.00000  Part-1  79.0    0.00000   
  133935999643424        1   2.00000   0.25000  Part-1  77.0    2.00000   
  133935999633296        1   2.25000   0.25000  Part-1  76.0    2.25000   
  133935999636032        1   2.50000   0.25000  Part-1  74.0    2.50000   
  133935999632288        1   2.75000   0.25000  Part-1  72.0    2.75000   
  133935999645296        2   3.00000   0.50000  Part-1  79.0    3.00000   
  133935999641984        2   3.50000   0.50000  Part-1  77.0    3.50000   
  133935999631904        2   4.00000   1.50000  Part-1  74.0    4.00000   
  133935999644768        2   5.50000   0.50000  Part-1  77.0    5.50000   
  133935999641072        3   6.00000   0.33333  Part-1  76.0    6.00000   
  133935999636464        3   6.33333   0.33333  Part-1  79.0    6.33333   
  1339359996403

Export estimated performance parameters to a kern or MEI file

SHOULD BE DONE DIRECTLY FROM A MODIFIED NMAT, EXPORTING TO A JSON FILE ISN'T AS EFFICIENT IF ALL FURTHER PROCESSING OF THE DATA IS WITHIN PYTHON.